<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/Ro_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-25_6syse
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-25_6syse
  Resolved https://github.com/huggingface/transformers to commit 638d49983f36af910934b38771b4e55c835c1774
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [27]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [30]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [32]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "ro", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [33]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [34]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 151MiB/s]


In [35]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [36]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="ro")
    return result["text"]

In [37]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [38]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [39]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.375, Precision: 0.7272727272727273, Recall: 0.6956521739130435, F1: 0.711111111111111
Processed 2 files, WER: 0.5, Precision: 0.47058823529411764, Recall: 0.47058823529411764, F1: 0.47058823529411764
Processed 3 files, WER: 0.6666666666666666, Precision: 0.375, Recall: 0.42857142857142855, F1: 0.39999999999999997
Processed 4 files, WER: 0.391304347826087, Precision: 0.5789473684210527, Recall: 0.5789473684210527, F1: 0.5789473684210527
Processed 5 files, WER: 0.6666666666666666, Precision: 0.5238095238095238, Recall: 0.55, F1: 0.5365853658536585
Processed 6 files, WER: 0.6451612903225806, Precision: 0.4444444444444444, Recall: 0.42857142857142855, F1: 0.43636363636363634
Processed 7 files, WER: 1.125, Precision: 0.1, Recall: 0.125, F1: 0.11111111111111112
Processed 8 files, WER: 0.5625, Precision: 0.4, Recall: 0.42857142857142855, F1: 0.4137931034482759
Processed 9 files, WER: 0.5, Precision: 0.5172413793103449, Recall: 0.5555555555555556, F1: 0.53571428571428